In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset

dataset = load_dataset("pile-of-law/pile-of-law", 'scotus_filings')

Loading Dataset Infos from C:\Users\BigLarry229\.cache\huggingface\modules\datasets_modules\datasets\pile-of-law--pile-of-law\c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
Overwrite dataset info from restored data version if exists.
Loading Dataset info from C:\Users\BigLarry229\.cache\huggingface\datasets/pile-of-law___pile-of-law/scotus_filings/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
Found cached dataset pile-of-law (C:/Users/BigLarry229/.cache/huggingface/datasets/pile-of-law___pile-of-law/scotus_filings/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60)
Loading Dataset info from C:/Users/BigLarry229/.cache/huggingface/datasets/pile-of-law___pile-of-law/scotus_filings/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60


In [3]:
print(dataset['train'].features)

{'text': Value(dtype='string', id=None), 'created_timestamp': Value(dtype='string', id=None), 'downloaded_timestamp': Value(dtype='string', id=None), 'url': Value(dtype='string', id=None)}


In [4]:
for field in dataset['train'][0].items():
  print(field)

('text', "b'IN THE SUPREME COURT OF THE UNITED STATES\\n\\nABELL, SHILO\\nPetitioner\\nvs.\\n\\nNo:\\n\\n21-0001\\n\\nUSA\\nWAIVER\\nThe Government hereby waives its right to file a response to the petition in this case,\\nunless requested to do so by the Court.\\n\\nELIZABETH B. PRELOGAR\\nActing Solicitor General\\nCounsel of Record\\nJuly 12, 2021\\ncc:\\nPETER CHARLES HORSTMANN\\nLAW OFFICES OF PETER CHARLES\\nHORSTMANN\\n450 LEXINGTON STREET\\nSUITE 101\\nNEWTON, MA 02466\\n\\n\\x0c'")
('created_timestamp', '07-12-2021')
('downloaded_timestamp', '10-19-2021')
('url', 'http://www.supremecourt.gov/DocketPDF/21/21-1/183636/20210712154623660_Waiver%20Letter%20-%2021-0001.pdf')


In [5]:
# This was a very dumb way of trying to find actual briefs within the dataset
for i in range(100):
  if 'brief' in dataset['train'][i]['text']:
    print("TEXT:")
    print(dataset['train'][i]['text'])
    print()

TEXT:
b'APPENDIX\n\n\x0cia\nTABLE OF CONTENTS\nPage\nAPPENDIX\nA\n\xe2\x80\x93\nJudgment\nwith\nUnpublished Disposition (D.C. Cir. Dec. 7,\n2020) ..........................................................................1a\nAPPENDIX B \xe2\x80\x93 Memorandum Opinion\nGranting\nPension\nBenefit\nGuaranty\nCorporation\xe2\x80\x99s\nMotion\nfor\nSummary\nJudgment and Denying Plaintiffs\xe2\x80\x99 Motion for\nSummary Judgment (D.D.C. June 11, 2018) ...........7a\nAPPENDIX C \xe2\x80\x93 Order Granting Pension\nBenefit Guaranty Corporation\xe2\x80\x99s Motion for\nSummary Judgment on Counts II through V\nand Dismissing Count VI of the First\nAmended Complaint (D.D.C. June 11, 2018) .........92a\nAPPENDIX D \xe2\x80\x93 Order Entering Final\nJudgment and Dismissing Count I of the\nFirst Amended Complaint (D.D.C. Aug. 29,\n2019) ........................................................................94a\nAPPENDIX E \xe2\x80\x93 Order Denying Petition for\nRehearing En Banc (D.C. Cir. Feb. 4,

In [6]:
print(len(dataset['train']))

47633


In [7]:
print(type(dataset))

<class 'datasets.dataset_dict.DatasetDict'>


# Saving the pile of law scotus_filings locally

In [8]:
import os
import pandas as pd
from datasets import load_dataset

# Create a subdirectory if it doesn't exist
subdirectory = 'scotus_filings'
if not os.path.exists(subdirectory):
    os.makedirs(subdirectory)

# CSV file to keep track of metadata
csv_filename = 'scotus_filings_metadata.csv'
csv_filepath = os.path.join(subdirectory, csv_filename)

# Initialize an empty DataFrame for the metadata
metadata_df = pd.DataFrame(columns=['Filename', 'Created Timestamp', 'Downloaded Timestamp', 'URL'])

# Function to save a scotus_filing and its metadata to the DataFrame
def save_scotus_filing_and_metadata_to_df(entry, scotus_filing_index, df):
    scotus_filing_content = entry['text']
    scotus_filing_name = f"scotus_filing_{scotus_filing_index}.txt"
    file_path = os.path.join(subdirectory, scotus_filing_name)

    # Save the scotus_filing content to a file
    with open(file_path, 'w') as file:
        file.write(scotus_filing_content)

    # Create a new DataFrame for the current entry and concatenate it with the existing one
    new_entry_df = pd.DataFrame([{'Filename': scotus_filing_name, 
                                  'Created Timestamp': entry['created_timestamp'], 
                                  'Downloaded Timestamp': entry['downloaded_timestamp'], 
                                  'URL': entry['url']}])
    return pd.concat([df, new_entry_df], ignore_index=True)

# Load the dataset
dataset = load_dataset("pile-of-law/pile-of-law", 'scotus_filings')

# Iterate through the dataset and save scotus_filings and metadata
for i, entry in enumerate(dataset['train']):
    text = entry['text']
    if 'BRIEF' in text:
        metadata_df = save_scotus_filing_and_metadata_to_df(entry, i, metadata_df)
        # Save to CSV every 50 iterations
        if i % 50 == 0 and i != 0:
            metadata_df.to_csv(csv_filepath, index=False, mode='a', header=False)  # Append to the CSV file
            metadata_df = metadata_df.iloc[0:0]  # Clear the DataFrame

# Save any remaining data in the DataFrame to the CSV file
if not metadata_df.empty:
    metadata_df.to_csv(csv_filepath, index=False, mode='a', header=False)

print(f"Scotus filings saved in '{subdirectory}', metadata saved in '{csv_filepath}'")


Loading Dataset Infos from C:\Users\BigLarry229\.cache\huggingface\modules\datasets_modules\datasets\pile-of-law--pile-of-law\c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
Overwrite dataset info from restored data version if exists.
Loading Dataset info from C:\Users\BigLarry229\.cache\huggingface\datasets/pile-of-law___pile-of-law/scotus_filings/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
Found cached dataset pile-of-law (C:/Users/BigLarry229/.cache/huggingface/datasets/pile-of-law___pile-of-law/scotus_filings/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60)
Loading Dataset info from C:/Users/BigLarry229/.cache/huggingface/datasets/pile-of-law___pile-of-law/scotus_filings/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60


Scotus filings saved in 'scotus_filings', metadata saved in 'scotus_filings\scotus_filings_metadata.csv'


# Moving the files without 'writ of certiorari' into a new directory

In [9]:
import os
import shutil
import time

# Original subdirectory containing the scotus_filings
original_subdirectory = 'scotus_filings'

# Paths for new directories at the same level as the notebook
with_certiorari_subdirectory = 'with_certiorari'
without_certiorari_subdirectory = 'without_certiorari'
os.makedirs(with_certiorari_subdirectory, exist_ok=True)
os.makedirs(without_certiorari_subdirectory, exist_ok=True)

# Function to categorize .txt files based on 'writ of certiorari'
def categorize_txt_files(base_directory, with_certiorari_subdirectory, without_certiorari_subdirectory):
    for filename in os.listdir(base_directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(base_directory, filename)

            # Check for 'writ of certiorari' in file content
            try:
                with open(file_path, 'r') as file:
                    content = file.read()
                if 'writ of certiorari' in content.lower():
                    target_directory = with_certiorari_subdirectory
                else:
                    target_directory = without_certiorari_subdirectory

                # Attempt to move the file to the appropriate directory
                try:
                    shutil.move(file_path, os.path.join(target_directory, filename))
                except PermissionError:
                    time.sleep(2)  # Wait for 2 seconds before trying again
                    shutil.move(file_path, os.path.join(target_directory, filename))
            except PermissionError as e:
                print(f"Error processing file {filename}: {e}")

# Categorize the .txt files
categorize_txt_files(original_subdirectory, with_certiorari_subdirectory, without_certiorari_subdirectory)
print(f".txt files categorized into '{with_certiorari_subdirectory}' and '{without_certiorari_subdirectory}'")


.txt files categorized into 'with_certiorari' and 'without_certiorari'


# Separating files with 'TABLE OF CONTENTS' from those without

In [10]:
import os
import shutil

# Path for the directory containing files without 'writ of certiorari'
without_certiorari_directory = 'without_certiorari'

# Paths for new directories at the same level as the notebook
with_toc_subdirectory = 'with_table_of_contents'
without_toc_subdirectory = 'without_table_of_contents'
os.makedirs(with_toc_subdirectory, exist_ok=True)
os.makedirs(without_toc_subdirectory, exist_ok=True)

# Function to categorize .txt files based on 'TABLE OF CONTENTS'
def categorize_files_based_on_toc(source_directory, with_toc_subdirectory, without_toc_subdirectory):
    for filename in os.listdir(source_directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(source_directory, filename)

            # Read and check the file content
            try:
                with open(file_path, 'r') as file:
                    content = file.read()
                if 'TABLE OF CONTENTS' in content:
                    target_directory = with_toc_subdirectory
                else:
                    target_directory = without_toc_subdirectory

                # Attempt to move the file
                try:
                    shutil.move(file_path, os.path.join(target_directory, filename))
                except PermissionError:
                    time.sleep(2)  # Wait for 2 seconds before trying again
                    shutil.move(file_path, os.path.join(target_directory, filename))
            except PermissionError as e:
                print(f"Error processing file {filename}: {e}")

# Categorize the .txt files
categorize_files_based_on_toc(without_certiorari_directory, with_toc_subdirectory, without_toc_subdirectory)
print(f".txt files categorized into '{with_toc_subdirectory}' and '{without_toc_subdirectory}'")


.txt files categorized into 'with_table_of_contents' and 'without_table_of_contents'


# Filter between single and multiple table of contents in the same file

In [11]:
import os
import shutil
import time

# Paths for new directories at the same level as the notebook
single_toc_subdirectory = 'ToC_filter/single_table_of_contents'
multiple_toc_subdirectory = 'ToC_filter/multiple_table_of_contents'
os.makedirs(single_toc_subdirectory, exist_ok=True)
os.makedirs(multiple_toc_subdirectory, exist_ok=True)

def count_and_categorize_toc(directory):
    single_toc_count = 0
    multiple_toc_count = 0

    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)

            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    toc_occurrences = content.count('TABLE OF CONTENTS')

                if toc_occurrences == 1:
                    single_toc_count += 1
                    target_directory = single_toc_subdirectory
                elif toc_occurrences > 1:
                    multiple_toc_count += 1
                    target_directory = multiple_toc_subdirectory

                # Move the file with retry logic
                max_attempts = 5
                for attempt in range(max_attempts):
                    try:
                        shutil.copy(file_path, os.path.join(target_directory, filename))
                        break  # If success, break out of the retry loop
                    except PermissionError:
                        if attempt < max_attempts - 1:
                            time.sleep(2)  # Wait for 2 seconds before trying again
                        else:
                            print(f"Failed to move file after {max_attempts} attempts: {filename}")

            except Exception as e:
                print(f"Error processing file {filename}: {e}")

    return single_toc_count, multiple_toc_count

# Directory containing files to be processed
with_toc_directory = 'with_table_of_contents'

# Execute the function
single_toc, multiple_toc = count_and_categorize_toc(with_toc_directory)
print(f"Files with a single 'TABLE OF CONTENTS': {single_toc}")
print(f"Files with multiple 'TABLE OF CONTENTS': {multiple_toc}")


Files with a single 'TABLE OF CONTENTS': 388
Files with multiple 'TABLE OF CONTENTS': 157


In [28]:
import pandas as pd
import os

# Load the metadata CSV into a DataFrame without headers
csv_path = 'scotus_filings_metadata.csv'
metadata_df = pd.read_csv(csv_path, header=None)

# Create a dictionary mapping filenames to URLs
file_url_map = dict(zip(metadata_df[0], metadata_df[3]))

# Directory containing the single 'TABLE OF CONTENTS' files
single_toc_directory = 'ToC_filter/single_table_of_contents'

# Filter the dictionary to include only files in the single_toc_directory
filtered_file_url_map = {filename: file_url_map[filename] for filename in os.listdir(single_toc_directory) if filename in file_url_map}

# 'filtered_file_url_map' contains the filenames and URLs of the PDFs in the single TOC directory
print(filtered_file_url_map)


{'scotus_filing_10264.txt': 'http://www.supremecourt.gov/DocketPDF/20/20-482/157135/20201008095426746_Adams%20and%20Boyle%20Appendix.pdf', 'scotus_filing_10482.txt': 'http://www.supremecourt.gov/DocketPDF/20/20-512/168477/20210208173849434_20210208%20Amicus%20Brief%20of%20States%20Supporting%20Petitioners.pdf', 'scotus_filing_10484.txt': 'http://www.supremecourt.gov/DocketPDF/20/20-512/168496/20210208191326511_210110a%20Amicus%20Brief%20for%20efiling.pdf', 'scotus_filing_10490.txt': 'http://www.supremecourt.gov/DocketPDF/20/20-512/171392/20210309152142101_Nos.%2020-512%2050-520%20-%20Brief%20of%20Amici%20Curiae.pdf', 'scotus_filing_10493.txt': 'http://www.supremecourt.gov/DocketPDF/20/20-512/171478/20210310124248942_20-512bsacUnitedStates.pdf', 'scotus_filing_10494.txt': 'http://www.supremecourt.gov/DocketPDF/20/20-512/171494/20210310133236766_Amicus%20Brief.pdf', 'scotus_filing_10498.txt': 'http://www.supremecourt.gov/DocketPDF/20/20-512/171571/20210310171709349_001_Brief_1707.pdf', '

# Using the URLs to download the PDFs to get a better look at the table of contents

In [29]:
import requests
import os

def download_pdf(url, directory, filename):
    # Ensure the filename ends with .pdf
    if not filename.endswith('.pdf'):
        filename = filename.replace('.txt', '.pdf')

    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(directory, filename), 'wb') as f:
            f.write(response.content)
    else:
        print(f"Failed to download {url}")
        return False
    return True

# Directory to save the downloaded PDFs
pdf_directory = 'downloaded_pdfs'
os.makedirs(pdf_directory, exist_ok=True)

# List to track failed downloads
failed_downloads = []

# Download each PDF
for filename, url in filtered_file_url_map.items():
    success = download_pdf(url, pdf_directory, filename)
    if not success:
        failed_downloads.append((filename, url))

print("Initial download attempt complete.")


Failed to download http://www.supremecourt.gov/DocketPDF/20/20A72/158727/20201024142754250_Circosta%20Motion%20For%20Leave%20to%20File%20Amicus%20Brief%20FINAL.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/20/20A87/161004/20201117154639544_20A87%20Amicus%20Brief%20First%20Liberty.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/20/20A87/161064/20201118104326342_20A87acAmericanMedicalAssociationAndMedicalSocietyOfTheStateOfNewYork.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/20/20A90/160920/20201117094308962_AU%20et%20al%20amicus%20motion%20with%20brief%20--%20Agudath%20Israel%20v%20Cuomo%20--%20final%20PDFA.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/20/20A90/161168/20201119102026299_20A90acTheAmericanMedicalAssociation.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/20/20A90/161412/20201120134349340_20A90%20Amicus%20First%20Liberty.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/20/20A90/161417/20201

Failed to download http://www.supremecourt.gov/DocketPDF/19/19-251/170598/20210301174346978_19-251%20-255%20tsac%20Protect%20the%201st%20and%20Pacific%20Research%20Institute.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-251/170342/20210301113245997_Brief%20of%20Amicus%20Curiae%20Institute%20for%20Justice%20in%20Support%20of%20Petitioners.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-251/170495/20210301150934618_AFP%20v%20Becerra%20merits.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-251/170548/20210301162005635_Buckeye%20Main%20Document%20E%20FILE%20Mar%201%202021.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-251/172982/20210325141442657_19%20251%2019%20255%20Brief%20on%20the%20Merits.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-251/173539/20210331145344824_19-251%20bsac%20Public%20Citizen.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-251/175665/202104141151238

Failed to download http://www.supremecourt.gov/DocketPDF/19/19-431/137423/20200309101217748_19-431%20Amicus%20Brief%20of%20Independent%20Womens%20Law%20Center.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-431/137463/20200309122650537_Little%20Sisters%20v%20Pennsylvanina-%20NCLA%20Amicus.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-431/137480/20200309132515310_19-431%2019-454%20tsac%20First%20Liberty%20Institute.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-431/137551/20200309164825846_Little%20Sisters%20Merits%20Amicus%20Brief%20TO%20FILE.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-431/137503/20200309144644738_Laycock%20Amicus%20Brief.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-431/137437/20200309110452036_19-431%2019-454%20TSAC%20ACLJ.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-431/137473/20200309125710656_19-431%20tsac%20Center%20for%20Constitutional%20

Failed to download http://www.supremecourt.gov/DocketPDF/19/19-840/143388/20200513104041259_PCORI%20Amicus.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-840/143391/20200513104715600_19-840-19-1019-AmericasHealthInsurancePlansAmicus.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-840/143462/20200513144457835_19-840%20-1019%20Amici%20BOM%20National%20Health%20Law%20Program.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-840/143449/20200513141543127_19-840%2010-1019%20CA%20v%20TEXAS%20S%20CT%20AMICUS%20BRIEF%20AARP.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-840/143451/20200513142138930_Nos.%2019-840.19-1019tsacFamiliesUSA.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-840/146374/20200625162030466_California%20v%20Texas%20PDF-A.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/19/19-840/146827/20200702111553329_19-840%20and%2019-1019%20ACLJ%20amicus%20brief.pdf
Failed to downlo

Failed to download http://www.supremecourt.gov/DocketPDF/19/19-1360/145045/20200605170259680_Cannon%20PFC%20APX.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/21/21-12/188552/20210825151910827_Cruz%20Opp%20to%20Mot%20to%20Aff%208.25.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/21/21A8/185735/20210804143559744_21A8%20Brief%20in%20Opposition%20to%20Application.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/21/21A22/187730/20210817152715798_Emergency%20Application%20for%20Writ%20of%20Injuction.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/21/21A77/195920/20211008140015228_Appendix%20to%20Peters%20Extension%20Application_Complete.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/21/21A85/196697/20211018145558016_U.S.%20v.%20Texas%20--%20motion%20for%20permission%20to%20file%20and%20brief%20--%202021.10.18.pdf
Failed to download http://www.supremecourt.gov/DocketPDF/21/21A85/196746/20211018171919718_21A85%20-%20United%20S

# Retrying on the failed files but using a VPN
## all but two worked, so I just manually downloaded them

In [31]:
def retry_failed_downloads(failed_list, directory):
    for filename, url in failed_list:
        print(f"Retrying {filename}")
        success = download_pdf(url, directory, filename)
        if success:
            print(f"Successfully downloaded {filename} on retry.")
        else:
            print(f"Failed again to download {filename}")

# Retry the failed downloads
retry_failed_downloads(failed_downloads, pdf_directory)

print("Retry attempt complete.")


Retrying scotus_filing_34345.txt
Successfully downloaded scotus_filing_34345.txt on retry.
Retrying scotus_filing_34397.txt
Successfully downloaded scotus_filing_34397.txt on retry.
Retrying scotus_filing_34402.txt
Successfully downloaded scotus_filing_34402.txt on retry.
Retrying scotus_filing_34414.txt
Successfully downloaded scotus_filing_34414.txt on retry.
Retrying scotus_filing_34420.txt
Successfully downloaded scotus_filing_34420.txt on retry.
Retrying scotus_filing_34423.txt
Successfully downloaded scotus_filing_34423.txt on retry.
Retrying scotus_filing_34425.txt
Successfully downloaded scotus_filing_34425.txt on retry.
Retrying scotus_filing_34427.txt
Successfully downloaded scotus_filing_34427.txt on retry.
Retrying scotus_filing_34438.txt
Successfully downloaded scotus_filing_34438.txt on retry.
Retrying scotus_filing_34444.txt
Successfully downloaded scotus_filing_34444.txt on retry.
Retrying scotus_filing_34453.txt
Successfully downloaded scotus_filing_34453.txt on retry.

Successfully downloaded scotus_filing_38525.txt on retry.
Retrying scotus_filing_38539.txt
Successfully downloaded scotus_filing_38539.txt on retry.
Retrying scotus_filing_38540.txt
Successfully downloaded scotus_filing_38540.txt on retry.
Retrying scotus_filing_38554.txt
Successfully downloaded scotus_filing_38554.txt on retry.
Retrying scotus_filing_38564.txt
Successfully downloaded scotus_filing_38564.txt on retry.
Retrying scotus_filing_38567.txt
Successfully downloaded scotus_filing_38567.txt on retry.
Retrying scotus_filing_38570.txt
Successfully downloaded scotus_filing_38570.txt on retry.
Retrying scotus_filing_38573.txt
Successfully downloaded scotus_filing_38573.txt on retry.
Retrying scotus_filing_38944.txt
Successfully downloaded scotus_filing_38944.txt on retry.
Retrying scotus_filing_38947.txt
Successfully downloaded scotus_filing_38947.txt on retry.
Retrying scotus_filing_38995.txt
Successfully downloaded scotus_filing_38995.txt on retry.
Retrying scotus_filing_38997.txt

Successfully downloaded scotus_filing_42922.txt on retry.
Retrying scotus_filing_42926.txt
Successfully downloaded scotus_filing_42926.txt on retry.
Retrying scotus_filing_42939.txt
Successfully downloaded scotus_filing_42939.txt on retry.
Retrying scotus_filing_43521.txt
Successfully downloaded scotus_filing_43521.txt on retry.
Retrying scotus_filing_43611.txt
Successfully downloaded scotus_filing_43611.txt on retry.
Retrying scotus_filing_44.txt
Successfully downloaded scotus_filing_44.txt on retry.
Retrying scotus_filing_44257.txt
Successfully downloaded scotus_filing_44257.txt on retry.
Retrying scotus_filing_44385.txt
Successfully downloaded scotus_filing_44385.txt on retry.
Retrying scotus_filing_44912.txt
Successfully downloaded scotus_filing_44912.txt on retry.
Retrying scotus_filing_45018.txt
Successfully downloaded scotus_filing_45018.txt on retry.
Retrying scotus_filing_45021.txt
Successfully downloaded scotus_filing_45021.txt on retry.
Retrying scotus_filing_45028.txt
Succe

Successfully downloaded scotus_filing_9389.txt on retry.
Retrying scotus_filing_9392.txt
Successfully downloaded scotus_filing_9392.txt on retry.
Retrying scotus_filing_9398.txt
Successfully downloaded scotus_filing_9398.txt on retry.
Retrying scotus_filing_9401.txt
Successfully downloaded scotus_filing_9401.txt on retry.
Retrying scotus_filing_9469.txt
Successfully downloaded scotus_filing_9469.txt on retry.
Retry attempt complete.


# Checking if 'Table of Authorities' is unique

In [34]:
def count_table_of_authorities(directory):
    single_toA_count = 0
    multiple_toA_count = 0

    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)

            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                toA_occurrences = content.count('TABLE OF AUTHORITIES')

            if toA_occurrences == 1:
                single_toA_count += 1

            elif toA_occurrences > 1:
                multiple_toA_count += 1

    print(f"single_toA_count: {single_toA_count}")
    print(f"multiple_toA_count: {multiple_toA_count}")
    
count_table_of_authorities("ToC_filter/single_table_of_contents")

single_toA_count: 98
multiple_toA_count: 225


# Ended up using it as a filter to more easily derive the table of contents

In [75]:
directory = "ToC_filter/single_table_of_contents"
with_ToA = []
without_ToA = []
for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)

            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                toA_occurrences = content.count('TABLE OF AUTHORITIES')

            if toA_occurrences == 0:
                without_ToA.append(filename)
            else:
                with_ToA.append(filename)

In [39]:
print(with_ToA)

['scotus_filing_10482.txt', 'scotus_filing_10484.txt', 'scotus_filing_10490.txt', 'scotus_filing_10493.txt', 'scotus_filing_10494.txt', 'scotus_filing_10498.txt', 'scotus_filing_10504.txt', 'scotus_filing_10509.txt', 'scotus_filing_10641.txt', 'scotus_filing_10667.txt', 'scotus_filing_10715.txt', 'scotus_filing_10760.txt', 'scotus_filing_10795.txt', 'scotus_filing_10804.txt', 'scotus_filing_10806.txt', 'scotus_filing_10814.txt', 'scotus_filing_10822.txt', 'scotus_filing_10825.txt', 'scotus_filing_10832.txt', 'scotus_filing_11058.txt', 'scotus_filing_11139.txt', 'scotus_filing_11707.txt', 'scotus_filing_11881.txt', 'scotus_filing_11921.txt', 'scotus_filing_12164.txt', 'scotus_filing_12169.txt', 'scotus_filing_12622.txt', 'scotus_filing_12870.txt', 'scotus_filing_12883.txt', 'scotus_filing_13155.txt', 'scotus_filing_14135.txt', 'scotus_filing_14781.txt', 'scotus_filing_14836.txt', 'scotus_filing_1551.txt', 'scotus_filing_1554.txt', 'scotus_filing_1557.txt', 'scotus_filing_16012.txt', 'sc

In [40]:
print(without_ToA)

['scotus_filing_10264.txt', 'scotus_filing_12663.txt', 'scotus_filing_12675.txt', 'scotus_filing_12717.txt', 'scotus_filing_13250.txt', 'scotus_filing_14585.txt', 'scotus_filing_14789.txt', 'scotus_filing_1508.txt', 'scotus_filing_1670.txt', 'scotus_filing_1675.txt', 'scotus_filing_17749.txt', 'scotus_filing_17810.txt', 'scotus_filing_17834.txt', 'scotus_filing_18703.txt', 'scotus_filing_18858.txt', 'scotus_filing_19241.txt', 'scotus_filing_20167.txt', 'scotus_filing_21111.txt', 'scotus_filing_21822.txt', 'scotus_filing_2195.txt', 'scotus_filing_22633.txt', 'scotus_filing_22740.txt', 'scotus_filing_23959.txt', 'scotus_filing_2415.txt', 'scotus_filing_24379.txt', 'scotus_filing_24716.txt', 'scotus_filing_2549.txt', 'scotus_filing_27600.txt', 'scotus_filing_29591.txt', 'scotus_filing_30269.txt', 'scotus_filing_30829.txt', 'scotus_filing_31337.txt', 'scotus_filing_33094.txt', 'scotus_filing_3373.txt', 'scotus_filing_34989.txt', 'scotus_filing_35245.txt', 'scotus_filing_3538.txt', 'scotus_

In [78]:
# import fitz
# import pandas as pd
# import os

# def extract_toc_toa(content):
#     start_pattern = "TABLE OF CONTENTS"
#     end_pattern = "TABLE OF AUTHORITIES"
#     start_index = content.find(start_pattern)
#     end_index = content.find(end_pattern)

#     if start_index != -1 and end_index != -1 and end_index > start_index:
#         return content[start_index:end_index]
#     else:
#         return None

# source_directory = "ToC_filter/single_table_of_contents"
# pdf_directory = 'downloaded_pdfs'
# csv_rows = []

# for filename in os.listdir(source_directory):
#     if filename.endswith('.txt') and 'TABLE OF AUTHORITIES' in open(os.path.join(source_directory, filename), 'r', encoding='utf-8').read():
#         pdf_filename = filename.replace('.txt', '.pdf')
#         pdf_path = os.path.join(pdf_directory, pdf_filename)
        
#         if os.path.exists(pdf_path):
#             doc = fitz.open(pdf_path)
#             content = ""
#             for page in doc:
#                 content += page.get_text()
#             doc.close()

#             extracted_text = extract_toc_toa(content)
#             if extracted_text:
#                 csv_rows.append([filename.replace('.txt', ''), extracted_text])

# toc_df = pd.DataFrame(csv_rows, columns=['Filing Name', 'Extracted Table of Contents'])
# toc_df.to_csv('extracted_toc.csv', index=False)

# print("CSV file created.")

# import fitz
# import pandas as pd
# import os
# import re

# def extract_toc_to_conclusion(content):
#     toc_pattern = r"TABLE OF CONTENTS"
#     conclusion_pattern = r"CONCLUSION"
#     toc_indices = [m.start() for m in re.finditer(toc_pattern, content)]
#     conclusion_indices = [m.start() for m in re.finditer(conclusion_pattern, content)]

#     # Skip files with multiple 'TABLE OF CONTENTS' or no 'CONCLUSION'
#     if len(toc_indices) != 1 or len(conclusion_indices) == 0:
#         return None

#     start_index = toc_indices[0]
#     end_index = next((i for i in conclusion_indices if i > start_index), None)

#     # Skip if 'CONCLUSION' is too far from 'TABLE OF CONTENTS'
#     if end_index and (end_index - start_index <= 500):
#         return content[start_index:end_index]
#     else:
#         return None

# source_directory = "ToC_filter/single_table_of_contents"
# pdf_directory = 'downloaded_pdfs'
# csv_rows = []

# for filename in os.listdir(source_directory):
#     if filename.endswith('.txt'):
#         pdf_filename = filename.replace('.txt', '.pdf')
#         pdf_path = os.path.join(pdf_directory, pdf_filename)

#         if os.path.exists(pdf_path):
#             doc = fitz.open(pdf_path)
#             content = ""
#             for page in doc:
#                 content += page.get_text()
#             doc.close()

#             extracted_text = extract_toc_to_conclusion(content)
#             if extracted_text:
#                 csv_rows.append([filename.replace('.txt', ''), extracted_text])

# toc_df = pd.DataFrame(csv_rows, columns=['Filing Name', 'Extracted Table of Contents'])
# toc_df.to_csv('extracted_toc.csv', index=False)

# print("CSV file created.")




CSV file created.


# Just checking if 'conclusion' was in all of these files
## - yes it is
## - also found there were multiple instances of 'conclusion', which is useful for later

In [70]:
directory = "ToC_filter/single_table_of_contents"
with_conclusion = []
without_conclusion = []
multiple_conclusions = 0
for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.exists(pdf_path):
            doc = fitz.open(pdf_path)
            content = ""
            for page in doc:
                content += page.get_text()
            doc.close()

        conclusion_occurrences = content.count('CONCLUSION')

        if conclusion_occurrences == 0:
            without_conclusion.append(filename)
        else:
            with_conclusion.append(filename)
            
        if conclusion_occurrences > 1:
            multiple_conclusions += 1

In [71]:
print(f"without_conclusion: {len(without_conclusion)}")
print(f"with_conclusion: {len(with_conclusion)}")
print(f"multiple_conclusions: {multiple_conclusions}")

without_conclusion: 0
with_conclusion: 388
multiple_conclusions: 388


# Extracting the table of contents
## - using the table of authorities as a possible marker

In [2]:
import fitz
import pandas as pd
import os
import re

def extract_toc_toa(content):
    toc_pattern = r"TABLE OF CONTENTS"
#     toa_pattern = r"TABLE OF AUTHORITIES(?![ .]{5,}\.{5,})"
    toa_pattern = r"TABLE OF AUTHORITIES(?![ .]{5,})"
    toc_indices = [m.start() for m in re.finditer(toc_pattern, content)]
    toa_indices = [m.start() for m in re.finditer(toa_pattern, content, re.DOTALL)]

    # Skip files with multiple 'TABLE OF CONTENTS' or no 'TABLE OF AUTHORITIES'
    if len(toc_indices) != 1 or len(toa_indices) == 0:
        return None

    start_index = toc_indices[0]
    end_index = next((i for i in toa_indices if i > start_index), None)

    return content[start_index:end_index]


source_directory = "ToC_filter/single_table_of_contents"
pdf_directory = 'downloaded_pdfs'
csv_rows = []

for filename in os.listdir(source_directory):
    if filename.endswith('.txt'):
        pdf_filename = filename.replace('.txt', '.pdf')
        pdf_path = os.path.join(pdf_directory, pdf_filename)

        if os.path.exists(pdf_path):
            doc = fitz.open(pdf_path)
            content = ""
            for page in doc:
                content += page.get_text()
            doc.close()

            extracted_text = extract_toc_toa(content)
            if extracted_text:
                csv_rows.append([filename.replace('.txt', ''), extracted_text])

toc_df = pd.DataFrame(csv_rows, columns=['Filing Name', 'Extracted Table of Contents'])
toc_df.to_csv('extracted_toc_v3.csv', index=False)

print("CSV file created.")


CSV file created.


## - also extracts the table of contents
## - but also uses CONCLUSION | Conclusion as a marker in addition to the table of authorities marker

In [7]:
import fitz
import pandas as pd
import os
import re

def extract_toc_toa(content):
    toc_pattern = r"TABLE OF CONTENTS"
    toa_pattern = r"TABLE OF AUTHORITIES(?![ .]{5,})"
    conclusion_pattern = r"(CONCLUSION|Conclusion)"
    
    def extract_text(start_pattern, end_pattern):
        start_indices = [m.start() for m in re.finditer(start_pattern, content)]
        end_indices = [m.start() for m in re.finditer(end_pattern, content, re.DOTALL)]

        if len(start_indices) != 1 or len(end_indices) == 0:
            return None

        start_index = start_indices[0]
        end_index = next((i for i in end_indices if i > start_index), None)

        return content[start_index:end_index]

    extracted_text = extract_text(toc_pattern, toa_pattern)

    # Use len() to compare lengths as integers
    if extracted_text and len(extracted_text.strip()) <= len('TABLE OF CONTENTS PagePage'):
        extracted_text = extract_text(toc_pattern, conclusion_pattern)

    return extracted_text

source_directory = "ToC_filter/single_table_of_contents"
pdf_directory = 'downloaded_pdfs'
csv_rows = []

for filename in os.listdir(source_directory):
    if filename.endswith('.txt'):
        pdf_filename = filename.replace('.txt', '.pdf')
        pdf_path = os.path.join(pdf_directory, pdf_filename)

        if os.path.exists(pdf_path):
            doc = fitz.open(pdf_path)
            content = ""
            for page in doc:
                content += page.get_text()
            doc.close()

            extracted_text = extract_toc_toa(content)
            if extracted_text:
                csv_rows.append([filename.replace('.txt', ''), extracted_text])

toc_df = pd.DataFrame(csv_rows, columns=['Filing Name', 'Extracted Table of Contents'])
# Changed the file name to not overwrite previous work
toc_df.to_csv('extracted_toc_v2.csv', index=False)

print("CSV file created.")

CSV file created.


# adding in the url to the CSV

In [16]:
import pandas as pd

# Load the CSV files into DataFrames
# toc_df = pd.read_csv('extracted_toc.csv')
toc_df = pd.read_csv('extracted_toc_v2.csv')
metadata_df = pd.read_csv('scotus_filings_metadata.csv', header=None)

# # Create a mapping from filing names to URLs
# # Assuming the filing name is in the first column (0) and the URL is in the fourth column (3)
# file_url_map = dict(zip(metadata_df[0], metadata_df[3]))

# # Add a new column for URLs in toc_df
# toc_df['URL'] = toc_df['Filing Name'].map(file_url_map)

# # Save the updated DataFrame
# toc_df.to_csv('extracted_toc_with_url_v2.csv', index=False)

# print("URLs added to extracted_toc_with_url_v2.csv.")

print(toc_df.head())
print(metadata_df.head())

# Assign column names to metadata_df for clarity
# column_names = ['Filing Name', 'Column2', 'Column3', 'URL']
# metadata_df.columns = column_names

# Remove .txt extension from filing names in metadata_df
metadata_df[0] = metadata_df[0].str.replace('.txt', '', regex=False).str.strip().str.lower()

# Ensure filing names in toc_df are in lower case and stripped
toc_df['Filing Name'] = toc_df['Filing Name'].str.strip().str.lower()

# Create the file_url_map
file_url_map = dict(zip(metadata_df[0], metadata_df[3]))

# Add URLs to toc_df
toc_df['URL'] = toc_df['Filing Name'].map(file_url_map)

# Check for entries in toc_df that didn't get a URL
unmatched_entries = toc_df[toc_df['URL'].isnull()]['Filing Name']
if not unmatched_entries.empty:
    print("Unmatched Entries:")
    print(unmatched_entries)
else:
    print("All entries matched.")

# Save the updated DataFrame
toc_df.to_csv('extracted_toc_with_url_v2.csv', index=False)

print("URLs added to extracted_toc_with_url_v2.csv.")


           Filing Name                        Extracted Table of Contents
0  scotus_filing_10482  TABLE OF CONTENTS \nPage \nTable of Contents ....
1  scotus_filing_10484  TABLE OF CONTENTS \nPage \nTABLE OF AUTHORITIE...
2  scotus_filing_10490  TABLE OF CONTENTS \nPage \nTABLE OF AUTHORITIE...
3  scotus_filing_10493  TABLE OF CONTENTS \nPage \nInterest of the Uni...
4  scotus_filing_10494  TABLE OF CONTENTS \n \nTABLE OF AUTHORITIES .....
                     0           1           2  \
0  scotus_filing_3.txt  08-02-2021  10-19-2021   
1  scotus_filing_4.txt  08-02-2021  10-19-2021   
2  scotus_filing_6.txt  08-16-2021  10-19-2021   
3  scotus_filing_7.txt  08-16-2021  10-19-2021   
4  scotus_filing_8.txt  08-16-2021  10-19-2021   

                                                   3  
0  http://www.supremecourt.gov/DocketPDF/21/21-2/...  
1  http://www.supremecourt.gov/DocketPDF/21/21-2/...  
2  http://www.supremecourt.gov/DocketPDF/21/21-2/...  
3  http://www.supremecourt.gov/Dock

# adding in the conclusion
## - finding the first instance by iterating backwards
## - not full proof but works most of the time

In [26]:
import fitz  # PyMuPDF
import pandas as pd
import os
import re

# def extract_conclusion_from_pdf(pdf_path):
#     # Open the PDF
#     doc = fitz.open(pdf_path)
#     conclusion_text = ""
#     conclusion_found = False

#     # Iterate backward through the PDF pages
#     for page_num in range(len(doc) - 1, -1, -1):
#         page = doc.load_page(page_num)
#         text = page.get_text()
#         if 'conclusion' in text.lower():
#             conclusion_index = text.lower().rfind('conclusion')
#             conclusion_text = text[conclusion_index:]
#             conclusion_found = True
#             break

#     doc.close()
#     return conclusion_text if conclusion_found else None

def extract_conclusion_from_pdf(pdf_path):
    # Open the PDF
    doc = fitz.open(pdf_path)
    conclusion_text = ""
    conclusion_found = False

    # Iterate backward through the PDF pages
    for page_num in range(len(doc) - 1, -1, -1):
        page = doc.load_page(page_num)
        text = page.get_text()

        # If conclusion not found yet, check for 'CONCLUSION' or 'Conclusion'
        if not conclusion_found:
            conclusion_index = max(text.rfind('CONCLUSION'), text.rfind('Conclusion'))
            if conclusion_index != -1:
                conclusion_text = text[conclusion_index:]
                conclusion_found = True

        # If conclusion is found, search for variations of 'Respectfully submitted'
        if conclusion_found:
            lower_text = text.lower()
            rs_index = lower_text.find('respectfully submitted')
            if rs_index != -1:
                # Check for variations: comma, period, or end of sentence/line
                next_char = lower_text[rs_index + len('respectfully submitted') : rs_index + len('respectfully submitted') + 1]
                if next_char in [',', '.', '', '\n']:
                    respectfully_submitted_index = rs_index
                    # Adjust the conclusion_text to end at 'Respectfully submitted'
                    conclusion_text = conclusion_text[:respectfully_submitted_index + len('Respectfully submitted') + len(next_char)]
                    break

    doc.close()
    return conclusion_text.strip() if conclusion_text else None



# Load the CSV file
toc_df = pd.read_csv('extracted_toc_with_url_v2.csv')

# Directory containing the downloaded PDFs
pdf_directory = 'downloaded_pdfs'

# Add a new column for Conclusions
toc_df['Conclusion'] = None

for index, row in toc_df.iterrows():
    filename = row['Filing Name'] + '.pdf'
    pdf_path = os.path.join(pdf_directory, filename)

    if os.path.exists(pdf_path):
        conclusion = extract_conclusion_from_pdf(pdf_path)
        toc_df.at[index, 'Conclusion'] = conclusion
    else:
        print(f"PDF not found for: {filename}")

# Save the updated DataFrame
toc_df.to_csv('extracted_toc_with_url_conclusion_v2.csv', index=False)

print("Updated CSV with conclusions.")


Updated CSV with conclusions.


# Filtering out some unwanted data from the Conclusion column

In [27]:
import pandas as pd

# Load the CSV file
toc_df = pd.read_csv('extracted_toc_with_url_conclusion_v2.csv')

# Filter out rows with empty or NaN 'Conclusion'
toc_df = toc_df[toc_df['Conclusion'].notna() & (toc_df['Conclusion'] != '')]

# Filter rows where the length of 'Conclusion' is less than "ConclusionConclusion"
min_length = len("ConclusionConclusion")
toc_df = toc_df[toc_df['Conclusion'].apply(lambda x: len(x) >= min_length)]

# Remove rows where 'Conclusion' contains 'Table of authorities'
toc_df = toc_df[~toc_df['Conclusion'].str.contains('Table of authorities', case=False)]

# Save the filtered DataFrame
toc_df.to_csv('filtered_extracted_toc_with_url_conclusion_v2.csv', index=False)

print("Filtered DataFrame saved as 'filtered_extracted_toc_with_url_conclusion_v2.csv'.")


Filtered DataFrame saved as 'filtered_extracted_toc_with_url_conclusion_v2.csv'.


# an attempt at a different method of looking at the data
## - tried this earlier on but it didn't work well

In [6]:
! pip install fpdf

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40705 sha256=619e0d31ec03b5cbde19a56a4c9e42e98f273370f7d2f6c5c61b1b25cfa626ee
  Stored in directory: c:\users\biglarry229\appdata\local\pip\cache\wheels\44\35\8b\86ce00cec7e4d13c5f189680ae0fa82f919bedc066c2cddae9
Successfully built fpdf


In [9]:
! pip install PyMuPDF IPython

     ---------------------------------------- 3.5/3.5 MB 37.3 MB/s eta 0:00:00
     --------------------------------------- 24.5/24.5 MB 65.2 MB/s eta 0:00:00


- Create a copy of each file and formating as a PDF
- didn't end up using this

In [11]:
import fitz
from IPython.display import display, Image

# Load the PDF
file_path = '20210805165140416_41247 pdf Daugherty.pdf'
doc = fitz.open(file_path)

# Extract and display the text content of each page
for i in range(len(doc)):
    page = doc.load_page(i)
    text = page.get_text()
    print(f"Page {i+1}:\n{text}\n{'='*40}\n")

Page 1:
 
No. 21-12 
In The 
Supreme Court of the United States 
--------------------------------- ♦ --------------------------------- 
FEDERAL ELECTION COMMISSION, 
Appellant,        
v. 
TED CRUZ FOR SENATE and 
RAFAEL EDWARD “TED” CRUZ, 
Appellees.        
--------------------------------- ♦ --------------------------------- 
On Appeal From The 
United States District Court 
For The District Of Columbia 
--------------------------------- ♦ --------------------------------- 
BRIEF AMICUS CURIAE OF 
INSTITUTE FOR FREE SPEECH 
IN SUPPORT OF APPELLEES 
--------------------------------- ♦ --------------------------------- 
DONALD A. DAUGHERTY, JR. 
INSTITUTE FOR FREE SPEECH 
1150 Connecticut Avenue, N.W. 
Suite 801 
Washington, DC 20036 
(202) 301-1664 
Facsimile: (202) 301-3399 
ddaugherty@ifs.org 
COCKLE LEGAL BRIEFS (800) 225-6964 
WWW.COCKLELEGALBRIEFS.COM 


Page 2:
i 
 
TABLE OF CONTENTS 
Page 
Interest of Amicus Curiae....................................  
1 
Summary of Argument .